In [10]:
#imports
%load_ext autoreload
%autoreload 2
import schillinger.harmony as SH
import schillinger.pitch as SP
import numpy as np
import itertools
import random

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
from music21 import *
%load_ext music21.ipython21

The music21.ipython21 extension is already loaded. To reload it, use:
  %reload_ext music21.ipython21


In [12]:
tonics = ["C","F","G"]

expansions = SP.expansions(tonics)

C3 = expansions[0]
Cm3 = expansions[1]
#C5 = expansions[2]
#C7 = expansions[4]

tonic_cycles = SP.pitch_cicles(tonics,[C3,Cm3],[1,1,1])
print(tonic_cycles) # All

['C', 'F', 'C', 'F', 'G', 'F', 'G', 'C', 'G', 'C']


In [13]:
#tonics to individual scales
scale_types = [scale.MinorScale]
scale_type = scale.MinorScale
all_root_scales = []
all_expansions = []
for e,i in enumerate(tonics):
    sc = scale_types[e%len(scale_types)] #choose different scale for every root
    my_temp_scale = sc(i)
    all_root_scales.append(my_temp_scale)
    scale_to_pitchnames = [str(p.name) for p in my_temp_scale.pitches[:-1]] # convert to pitchnames
    all_expansions.append(SP.expansions(scale_to_pitchnames))
all_root_scales, #all_expansions

([<music21.scale.MinorScale C minor>,
  <music21.scale.MinorScale F minor>,
  <music21.scale.MinorScale G minor>],)

In [14]:
tonic_sequence = tonic_cycles
durations = [1] # hardcode

cadences = []
for exp in all_expansions:
    cadence = SH.diatonic_cadences(exp)
    cadences.append(cadence)
    #print(cadence)
    
#tonic_sequence

In [15]:
filled_harmonies = []
filled_melody = []
#filled_bass = []
voices = 4
for i, rs in enumerate(tonic_sequence):
    
    #tonics
    current_harmony_idx = tonics.index(rs)
    my_scale = all_root_scales[current_harmony_idx]
    expansions = all_expansions[current_harmony_idx] # so
    chorified_scale = SH.chordify_scale(expansions[1],voices)
    harmonized_bass = SH.harmonize_note(rs, chorified_scale, i%voices, randomized=False)
    filled_harmonies.append(harmonized_bass)
    
    filled_melody.append(random.choice(random.choice(expansions)))
    
    ######
    #between tonics # experimental
    next_harmony = tonic_sequence[(i+1)%len(tonic_sequence)]
    next_harmony_idx = tonics.index(next_harmony)
    my_scale = all_root_scales[next_harmony_idx]
    expansions = all_expansions[next_harmony_idx]
    chorified_scale = SH.chordify_scale(expansions[1],voices)
    
    C7,C5,C3 = expansions[0],expansions[2], expansions[4]
    #print(next_harmony)
    my_root_cycles = SP.pitch_cicles(expansions[0],[C3,C5,C7],[2,1,1,2])#to much but wurscht!
    
    #print(my_root_cycles)
    
    for d in range(0, durations[i%len(durations)]-1):
        temp_bass = my_root_cycles[(d+1)%len(my_root_cycles)]
        #print(temp_bass)
        harmonized_bass = SH.harmonize_note(temp_bass, chorified_scale, d%voices, randomized=False)
        filled_harmonies.append(harmonized_bass)
        
        filled_melody.append(random.choice(random.choice(expansions)))
    
        
print(filled_harmonies)


[['C', 'E-', 'G', 'B-'], ['D-', 'F', 'A-', 'C'], ['F', 'A-', 'C', 'E-'], ['G', 'B-', 'D-', 'F'], ['G', 'B-', 'D', 'F'], ['D-', 'F', 'A-', 'C'], ['C', 'E-', 'G', 'B-'], ['D', 'F', 'A-', 'C'], ['G', 'B-', 'D', 'F'], ['A-', 'C', 'E-', 'G']]


In [16]:
#old
'''filled_harmonies = []
filled_bass = []
for i, rs in enumerate(root_sequence):
    
    #fill in the first or tonic note
    current_harmony = root_sequence[(i)%len(root_sequence)]
    current_harmony_idx = tonics.index(current_harmony)
    my_scale = all_root_scales[current_harmony_idx]
    expansions = all_expansions[current_harmony_idx] # so
    
    chorified_scale = SPG.chordify_scale(expansions[1],4)
    print(rs,chorified_scale)
    harmonized_bass = SH.harmonize_bass([rs], chorified_scale)
    print(harmonized_bass)
    filled_harmonies.append(harmonized_bass[0][1])
    #filled_harmonies.append(harmonized_bass[0][1])
    
    filled_bass.append(harmonized_bass[0][1][0])
    
    
    next_harmony = root_sequence[(i+1)%len(root_sequence)]
    next_harmony_idx = tonics.index(next_harmony)
    my_scale = all_root_scales[next_harmony_idx]
    # descision if cadence or whole expansion!
    expansions = all_expansions[next_harmony_idx]
    
    chorified_scale = SPG.chordify_scale(expansions[1],4)
    
    C7,C5,C3 = expansions[0],expansions[2], expansions[4]
    my_root_cycles = SP.pitch_cicles(next_harmony,[C3,C7,C5],[1,1,2,1])#to much but wurscht!
    
    
    
    for d in range(0, durations[i]-1):
        temp_bass = my_root_cycles[(d+1)%len(my_root_cycles)]
        harmonized_bass = SPG.harmonize_bass([temp_bass], chorified_scale)
        
        filled_harmonies.append(harmonized_bass[0][1])
        filled_bass.append(harmonized_bass[0][1][0])
        #print(harmonized_bass[0][1], my_cadences[0])
        
#print(filled_harmonies)''';

In [17]:
def showNotes(note_sequece, time_meter, duration_divider):
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    s = stream.Score(id='mainScore')
    p0 = stream.Part(id='part0')
    p0.timeSignature = meter.TimeSignature(TS)
    for i, notes in enumerate(note_sequece):
        #print(notes)
        c = chord.Chord(notes)
        #print(c,c.root(),c.third)
        c.quarterLength = 1 * duration_divider
        c = c.semiClosedPosition(inPlace=False)
        chords =  arpeggiator(c, es, rhythm, duration_divider)
        #c.octave = 3
        #c.pitches[2].octave = 3
        #c.pitches[0].octave = 1
        #c.inversion(0)
        p0.append(c)
    s.insert(0, p0)
    return s

In [18]:
# notes harmony 

def arpeggiator(c, es,rhythm, duration_divider):
    chords = []
    for i, seq in enumerate(c):
                
        nota = seq
        nota.duration.quarterLength = (rhythm[es%len(rhythm)]/len(c)) * duration_divider
        chords.append(nota)

    nota = pitch.Pitch(c.pitches[0])
    
    nota.octave = nota.octave + 1
    nota = note.Note(nota.nameWithOctave)
    nota.duration.quarterLength = (rhythm[es%len(rhythm)]/len(c)) * duration_divider
    chords.append(nota)
    #octave adjust
    #print(chords[-1].pitch.midi - chords[-2].pitch.midi, chords[-1])
    if chords[-2].pitch.midi - chords[-3].pitch.midi >= 6:
        chords[-2].octave = chords[-2].octave + 1
        
    if chords[-1].pitch.midi - chords[-2].pitch.midi >= 12:
        chords[-1].octave = chords[-1].octave -1
        #print("yes", chords[-1], chords[-2])
    
    ## last note repeat the second
    nota = pitch.Pitch(c.pitches[2])
    nota.octave = nota.octave
    nota = note.Note(nota.nameWithOctave)
    nota.duration.quarterLength = (rhythm[es%len(rhythm)]/len(c)) * duration_divider
    chords.append(nota)
    return chords

def showHarmony(note_sequence, melody_seq, rhythm, length, time_meter, duration_divider, arp=True,bass=True, drop2 = False):
    TS = str(time_meter[0])+'/'+str(time_meter[1])
    
    s = stream.Score(id='mainScore')
    p0 = stream.Part(id='melody')
    p0.timeSignature = meter.TimeSignature(TS)
    p1 = stream.Part(id='chords')
    p1.timeSignature = meter.TimeSignature(TS)
    p2 = stream.Part(id='bass')
    p2.timeSignature = meter.TimeSignature(TS)
    
    melody_iterator = 0
    for es, sequences in enumerate(note_sequence):
        
        melody = []
        chords = []
        bass = []
        
        
        ### chords
        chord_seq = sequences
        c = chord.Chord(chord_seq)
        
        #test_chord = chord.Chord(test_chord)
        c.root = sequences[0]
        c.duration.quarterLength = rhythm[es%len(rhythm)] * duration_divider
        #c = c.sortDiatonicAscending() # for better voice leadng but not real!
        c = c.semiClosedPosition(inPlace=False,forceOctave=4)
        
        if drop2:
            try:
                #c.pitches[0].octave = c.pitches[0].octave-1 # drop2
                #c.pitches[-2].octave = c.pitches[-2].octave+2 # drop4
                pass
            except:
                pass
        
        if arp:
            chords =  arpeggiator(c, es, rhythm, duration_divider)
            #chords.append(nota)
            
        else:
            chords.append(c)
            
        #MELODY
        # get chord durations and apply on melody (for test purpose)
        '''dur = 0
        for d in chords:
            dur = dur + d.duration.quarterLength
        #print(dur)
        nota = pitch.Pitch(melody_seq[es])
        nota.octave = nota.octave
        nota = note.Note(nota.nameWithOctave)
        nota.duration.quarterLength = dur/2
        melody.append(nota)
        #melody 2nd tone
        ###'''
        #melody
        # get chord durations and apply on melody (for test purpose)
        dur = 0
        for d in chords:
            dur = dur + d.duration.quarterLength
        #print(melody_seq, es)
        nota = pitch.Pitch(random.choice(sequences))
        nota.octave = 4
        nota = note.Note(nota.nameWithOctave)
        nota.duration.quarterLength = dur
        melody.append(nota)
        
        
        
        #BASS
        # get chord durations and apply on melody (for test purpose)
        dur = 0
        for d in chords:
            dur = dur + d.duration.quarterLength
        #print(melody_seq, es)
        nota = pitch.Pitch(sequences[0])
        nota.octave = 2
        nota = note.Note(nota.nameWithOctave)
        nota.duration.quarterLength = dur
        bass.append(nota)
        
        
        
        p0.append(melody)

        p1.append(chords)
        p2.append(bass)
    
    #s.insert(0, p0)
    s.insert(0, p1)
    s.insert(0, p2)
    s = s.transpose(0)
    return s


In [19]:
#s = showHarmony(filled_harmonies,[4,4],2)
s = showHarmony(filled_harmonies, filled_melody,[1],len(filled_harmonies),[4,4],1, arp=True)
s.show('musicxml')

In [192]:
return_value = [list(itertools.chain(*filled_harmonies))][0]
#print(return_value)
#huhu